In [2]:
import pandas as pd
import numpy as np
import json
import requests
from bs4 import BeautifulSoup
import datetime

--------------
<h3>Tienda BNA

In [3]:
start = datetime.datetime.now()
print("Hora de inicio:",start.time())

Hora de inicio: 01:39:07.542425


In [4]:
url = "https://www.tiendabna.com.ar/catalog"

In [6]:
page = requests.get(url)
cuerpo = BeautifulSoup(page.content)
categories_list = cuerpo.findAll("a",{"class",""})

In [16]:
cuerpo

<!DOCTYPE html>
<html><head>
<meta charset="utf-8"/>
<title>Tienda Banco Nacion Argentina</title>
<base href="/"/>
<script data-cfasync="false" type="text/javascript">

    var dataLayer = dataLayer || [];

    if (global === undefined) { var global = window; }

  </script>
<meta content="initial-scale=1, maximum-scale=1" name="viewport"/>
<link href="/assets/stores/current_store/images/favicon.ico" rel="icon" type="image/x-icon"/>
<link href="styles.df0fb366a07edfa14aaf.css" rel="stylesheet"/><style ng-transition="avenidafrontend">@import url("https://fonts.googleapis.com/icon?family=Material+Icons");
























@font-face {
  font-family: swiper-icons;
  src: url("data:application/font-woff;charset=utf-8;base64, d09GRgABAAAAAAZgABAAAAAADAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABGRlRNAAAGRAAAABoAAAAci6qHkUdERUYAAAWgAAAAIwAAACQAYABXR1BPUwAABhQAAAAuAAAANuAY7+xHU1VCAAAFxAAAAFAAAABm2fPczU9TLzIAAAHcAAAASgAAAGBP9V5RY21hcAAAAkQAAACIAAABYt6F0cBjdnQgAAACzAAAAAQAAAAEABEBRGdhc3AAAAWYAAAACAA

In [9]:
list_categories = []

for categories in categories_list:
        new_href = categories.get("href")
        new_href="https://www.tiendabna.com.ar"+new_href
        new_cat_title = categories.text.strip()
#         if new_herf != "None":
        list_categories.append([new_href,new_cat_title])
    
list_categories = pd.DataFrame(list_categories,columns = ['url','category'])

In [8]:
list_categories.drop_duplicates(subset="url",inplace=True)
print(list_categories.shape)
list_categories.head()

(0, 2)


,url,category


In [11]:
data_productos = []
for x in list_categories.index:
    try:
        url = list_categories.loc[x,'url']
        category = list_categories.loc[x,'category']
        page = requests.get(url)
        cuerpo = BeautifulSoup(page.content)

        last_page= int(cuerpo.find("div",{"class","catalog-bottom"}).findAll("a")[-1].text)
    except:
        pass
    
    for page_num in range(0,last_page):
        try:
            page_num=str(page_num+1)
            url_num = url+"?p="+str(page_num)
            page = requests.get(url_num)
            cuerpo = BeautifulSoup(page.content)
            lista_productos = cuerpo.findAll("article")
            print("Categoría "+category,". Página "+str(page_num),". URL:",url_num)
        except:
            pass

        for cuerpo_producto in lista_productos:
            try:
                url_producto = "https://www.tiendabna.com.ar/"+cuerpo_producto.find("a").get("href")
                nombre_producto = cuerpo_producto.find("h6").text
                precio_producto = cuerpo_producto.find("span",{"class","price sale-price"}).text.strip()
                data_productos.append([category,url_num,url_producto,nombre_producto,precio_producto])
            except:
                pass

In [12]:
data_productos_final = pd.DataFrame(data_productos,columns=["category","url","url_producto","nombre_producto","precio_producto"])

In [13]:
print(data_productos_final.shape)
data_productos_final.head()

(0, 5)


,category,url,url_producto,nombre_producto,precio_producto


In [14]:
data_productos_final.to_excel("TiendaBNA_productos.xlsx")
data_productos_final.to_csv("TiendaBNA_productos.csv")

ModuleNotFoundError: No module named 'openpyxl'

In [15]:
end = datetime.datetime.now()
print("Hora de inicio:",start.time())
print("Hora de finalización:",end.time())
print("Tiempo de procesamiento:",round(((end-start).seconds)/60,1),"minutos")

Hora de inicio: 01:39:07.542425
Hora de finalización: 02:05:30.374105
Tiempo de procesamiento: 26.4 minutos
